# Advanced AERMOD Features

This notebook covers advanced PyAERMOD features:

1. Background concentrations
2. Deposition modeling
3. EVENT processing
4. NO2 chemistry options
5. Source group management

**Requirements:**
- PyAERMOD installed (`pip install pyaermod`)

In [ ]:
from pyaermod.input_generator import (
    AERMODProject,
    BackgroundConcentration,
    BackgroundSector,
    CartesianGrid,
    ChemistryMethod,
    ChemistryOptions,
    ControlPathway,
    DepositionMethod,
    DiscreteReceptor,
    EventPathway,
    EventPeriod,
    GasDepositionParams,
    MeteorologyPathway,
    OutputPathway,
    OzoneData,
    ParticleDepositionParams,
    PointSource,
    ReceptorPathway,
    SourceGroupDefinition,
    SourcePathway,
)

print("All advanced feature imports successful!")

## 1. Background Concentrations

Add ambient background levels to AERMOD predictions. Three modes:
- **Uniform**: single value for all periods
- **Period-specific**: different values per averaging period
- **Sector-dependent**: varying by wind direction sector

In [ ]:
# Uniform background
bg_uniform = BackgroundConcentration(
    mode="uniform",
    uniform_value=5.0,  # ug/m3 for all periods
)

# Sector-dependent background
bg_sector = BackgroundConcentration(
    mode="sector",
    sectors=[
        BackgroundSector(sector_id=1, start_direction=0, end_direction=90, value=3.0),
        BackgroundSector(sector_id=2, start_direction=90, end_direction=180, value=5.0),
        BackgroundSector(sector_id=3, start_direction=180, end_direction=270, value=4.0),
        BackgroundSector(sector_id=4, start_direction=270, end_direction=360, value=6.0),
    ],
)

# Apply to source pathway
sources = SourcePathway()
sources.add_source(PointSource(
    source_id="STACK1", x_coord=0, y_coord=0, base_elevation=0,
    stack_height=50, stack_temp=400, exit_velocity=15,
    stack_diameter=2, emission_rate=1.5,
))
sources.background = bg_uniform

print("Background concentration configured")
print(f"  Mode: {bg_uniform.mode}")
print(f"  Value: {bg_uniform.uniform_value} ug/m3")

## 2. Deposition Modeling

Model dry and/or wet deposition of pollutants using gas or particle parameters.

In [ ]:
# Gas deposition parameters
gas_dep = GasDepositionParams(
    diffusivity=0.15,          # cm2/s
    alpha_r=40.0,              # dimensionless effective Henry's law constant
    reactivity=8.0,            # reactivity factor
    henry_constant=1000.0,     # Pa m3/mol
)

# Particle deposition parameters
particle_dep = ParticleDepositionParams(
    diameters=[1.0, 2.5, 10.0],        # microns
    mass_fractions=[0.3, 0.5, 0.2],     # sum to 1.0
    densities=[2.0, 2.0, 2.5],          # g/cm3
)

# Apply to a source — deposition_method is a (DepositionMethod, float) tuple
dep_source = PointSource(
    source_id="DEP_STK", x_coord=0, y_coord=0, base_elevation=0,
    stack_height=30, stack_temp=350, exit_velocity=12,
    stack_diameter=1.5, emission_rate=2.0,
    gas_deposition=gas_dep,
    deposition_method=(DepositionMethod.DRYDPLT, 0.0),
)

print("Deposition source configured")
print(f"  Method: {dep_source.deposition_method}")
print(f"  Gas diffusivity: {gas_dep.diffusivity} cm2/s")

## 3. EVENT Processing

Analyze specific dates/hours at specific receptors.

In [ ]:
# Define events for specific receptor locations at specific times
events = EventPathway(
    events=[
        EventPeriod(
            event_name="PEAK_HR",
            source_group="ALL",
            averaging_period="1",
            start_date="20010315",
            start_hour=14,
            end_date="20010315",
            end_hour=14,
        ),
        EventPeriod(
            event_name="CALM_HR",
            source_group="ALL",
            averaging_period="1",
            start_date="20010720",
            start_hour=6,
            end_date="20010720",
            end_hour=6,
        ),
    ]
)

print(f"Event pathway configured with {len(events.events)} events:")
for evt in events.events:
    print(f"  {evt.event_name}: {evt.start_date} hour {evt.start_hour}")

## 4. NO2 Chemistry Options

Model NO2 formation from NOx emissions using EPA-approved methods.

In [ ]:
# PVMRM method with ozone data file
chemistry = ChemistryOptions(
    method=ChemistryMethod.PVMRM,
    default_no2_ratio=0.5,  # default equilibrium ratio
    ozone_data=OzoneData(
        ozone_file="ozone_data.dat",  # hourly ozone concentrations
    ),
)

# Apply to control pathway
control = ControlPathway(
    title_one="NO2 Chemistry Example",
    pollutant_id="NO2",
    averaging_periods=["1", "ANNUAL"],
    chemistry=chemistry,
)

print("Chemistry options configured")
print(f"  Method: {chemistry.method}")
print(f"  Default NO2/NOx ratio: {chemistry.default_no2_ratio}")
print(f"  Ozone file: {chemistry.ozone_data.ozone_file}")

## 5. Source Group Management

Group sources for separate output analysis.

In [ ]:
# Create multiple sources
sources = SourcePathway()
sources.add_source(PointSource(
    source_id="BOILER1", x_coord=0, y_coord=0, base_elevation=0,
    stack_height=40, stack_temp=450, exit_velocity=20,
    stack_diameter=2, emission_rate=3.0,
))
sources.add_source(PointSource(
    source_id="BOILER2", x_coord=50, y_coord=0, base_elevation=0,
    stack_height=40, stack_temp=450, exit_velocity=20,
    stack_diameter=2, emission_rate=2.5,
))
sources.add_source(PointSource(
    source_id="FLARE1", x_coord=100, y_coord=100, base_elevation=0,
    stack_height=60, stack_temp=1200, exit_velocity=25,
    stack_diameter=1, emission_rate=0.5,
))

# Define groups
sources.group_definitions = [
    SourceGroupDefinition(
        group_name="BOILERS",
        member_source_ids=["BOILER1", "BOILER2"],
        description="Combined boiler emissions",
    ),
    SourceGroupDefinition(
        group_name="FLARES",
        member_source_ids=["FLARE1"],
        description="Emergency flare",
    ),
]

print("Source groups defined:")
for grp in sources.group_definitions:
    print(f"  {grp.group_name}: {grp.member_source_ids} - {grp.description}")

## 6. Complete Example

Putting it all together — a project with background + chemistry + source groups:

In [ ]:
# Apply background to sources
sources.background = BackgroundConcentration(mode="uniform", uniform_value=10.0)

# Build project
receptors = ReceptorPathway()
receptors.add_cartesian_grid(CartesianGrid(
    x_init=-500, x_num=11, x_delta=100,
    y_init=-500, y_num=11, y_delta=100,
))

met = MeteorologyPathway(surface_file="met.sfc", profile_file="met.pfl")
output = OutputPathway(receptor_table=True, max_table_rank=10)

project = AERMODProject(
    control=control,
    sources=sources,
    receptors=receptors,
    meteorology=met,
    output=output,
)

# Preview generated input
inp = project.to_aermod_input(validate=False, check_files=False)
print("Generated input preview (first 40 lines):")
print("=" * 70)
for line in inp.split("\n")[:40]:
    print(line)
print("...")

## Summary

| Feature | Key Classes | Keywords Generated |
|---------|------------|-------------------|
| Background | `BackgroundConcentration`, `BackgroundSector` | `BACKGRND`, `BGSECTOR` |
| Deposition | `GasDepositionParams`, `ParticleDepositionParams` | `GASDEPOS`, `PARTDIAM`, `MASSFRAX`, `PARTDENS` |
| Events | `EventPathway`, `EventPeriod` | `EVENTPER` |
| Chemistry | `ChemistryOptions`, `OzoneData` | `MODELOPT PVMRM`, `O3VALUES`, `OZONEFIL` |
| Groups | `SourceGroupDefinition` | `SRCGROUP` |